In [21]:
library(limma)
library(dplyr) 
library(ggplot2)
library(readxl)
library(viridis)

In [22]:
# Cargar la matriz de expresión
expr_file <- "/home/mlopez/Desktop/alzheimer/data/GSE138260/GSE138260_series_matrix.txt.gz"
raw_data <- read.delim(gzfile(expr_file), comment.char = "!", header = TRUE, row.names = 1)
raw_data <- as.data.frame(raw_data)

# Cargar los metadata
metadata <- read.csv("/home/mlopez/Desktop/alzheimer/results/GSE138260/GSE138260_metadata.csv", stringsAsFactors = FALSE)
rownames(metadata) <- metadata$Accession

# Leer líneas completas como texto
ann_lines <- readLines("/home/mlopez/Desktop/alzheimer/data/GSE138260/annotation_lines.txt")

# Separar por tabulador (puede haber columnas vacías, así que no usar sep fijo)
ann_split <- strsplit(ann_lines, "\t")

# Ver longitud máxima de columnas
max(sapply(ann_split, length))  # te dará algo como 10

# Convertir a data.frame con relleno
ann_df <- do.call(rbind, lapply(ann_split, function(x) {
  length(x) <- 10  # Asegurar que todas las filas tengan 10 columnas
  return(x)
}))

# Convertir a data.frame

ann_df <- as.data.frame(ann_df, stringsAsFactors = FALSE)

# Renombrar columnas (ajustar si alguna no calza)
colnames(ann_df) <- c("ProbeID", "ProbeName", "IsControl", "Empty1", "Empty2",
                      "GeneSymbol", "GeneName", "Cytoband", "Annotation", "Sequence")

# Asegúrate de que GeneName no tenga NA
ann_df <- ann_df[!is.na(ann_df$GeneName), ]

[1] 16

In [23]:
raw_data$ProbeID <- rownames(raw_data)
 
raw_data_annot <- merge(raw_data, ann_df, by.x="ProbeID", by.y="ProbeID")

In [24]:
# Primero, asegurémonos de que no haya duplicados en el merge por GeneName
# El merge va a añadir las anotaciones, pero si hay duplicados, solo tomamos el primer valor por GeneName
raw_data_annot <- raw_data_annot[!duplicated(raw_data_annot$GeneName), ]

# Establecer GeneName como rownames
rownames(raw_data_annot) <- raw_data_annot$GeneName

In [25]:
raw_data_annot <- data.frame(raw_data_annot[, grep("GSM",colnames(raw_data_annot))])
expr_matrix <- raw_data_annot[, sapply(raw_data_annot, is.numeric)]

In [26]:
pdf("/home/mlopez/Desktop/alzheimer/results/GSE138260/boxplot_raw_expr_GSE138260.pdf")
# Boxplot por muestra (cada columna es una muestra)
boxplot(expr_matrix,
        main = "Boxplot por muestra",
        xlab = "Muestras",
        ylab = "Expresión (log2?)",
        las = 2,           # Rota nombres del eje x
        col = "lightblue",
        outline = FALSE)   # No mostrar outliers para una vista más clara
dev.off()

pdf 
  2

In [27]:
# Crear un objeto tipo ExpressionSet si quieres (opcional)
# o trabajar directamente con la matriz
expr_matrix_norm <- normalizeBetweenArrays(as.matrix(expr_matrix), method = "quantile")

pdf("/home/mlopez/Desktop/alzheimer/results/GSE138260/boxplot_norm_expr_GSE138260.pdf")
boxplot(expr_matrix_norm,
        main = "Boxplot tras normalización (cuantiles)",
        xlab = "Muestras",
        ylab = "Expresión normalizada",
        las = 2,
        col = "lightgreen",
        outline = FALSE)
dev.off()

pdf 
  2

In [28]:
# Transponer para que las filas sean muestras
pca <- prcomp(t(expr_matrix_norm), scale. = TRUE)

pca_df <- as.data.frame(pca$x[, 1:2])  # Primeras 2 PCs
pca_df$SampleID <- rownames(pca_df)
pca_df <- merge(pca_df, metadata, by.x = "SampleID", by.y="Accession")

pdf("/home/mlopez/Desktop/alzheimer/results/GSE138260/pca_norm_expr_GSE138260.pdf")
ggplot(pca_df, aes(x = PC1, y = PC2, color = Group)) +
  geom_point(size = 3, alpha = 0.8) +
  theme_minimal() +
  labs(title = "PCA of Expression Data",
       subtitle = "Colored by AD/Control, shaped by Brain Region",
       x = paste0("PC1 (", round(summary(pca)$importance[2,1] * 100, 1), "%)"),
       y = paste0("PC2 (", round(summary(pca)$importance[2,2] * 100, 1), "%)")) +
  theme(axis.text = element_text(size=10),
        plot.title = element_text(face="bold", size=14),
        legend.title = element_text(size=12))
dev.off()

pdf 
  2

In [29]:
# Asegurarte que el orden de las muestras en metadata coincida con las columnas de la matriz de expresión
metadata <- metadata[colnames(expr_matrix_norm), ]

# Crear factor de grupo
group <- factor(metadata$Group, levels = c("CONTROL", "AD"))

# Crear matriz de diseño
design <- model.matrix(~ 0 + group)
colnames(design) <- levels(group)

# Definir contraste: AD - CONTROL
contrast_matrix <- makeContrasts(AD_vs_CONTROL = AD - CONTROL, levels = design)

# Ajustar el modelo lineal
fit <- lmFit(expr_matrix_norm, design)

# Aplicar el contraste
fit2 <- contrasts.fit(fit, contrast_matrix)

# Aplicar moderación de varianza con eBayes
fit2 <- eBayes(fit2)

results <- topTable(fit2, coef = "AD_vs_CONTROL", number = Inf, adjust = "fdr")

In [30]:
AD_T4_targets <- read_excel("/home/mlopez/Desktop/alzheimer/data/AD_T4_targets.xlsx")

New names:
• `` -> `...1`


In [32]:
na.omit(results[AD_T4_targets$Prot,])

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SOX2,-0.1330562676,8.726533,-0.759406584,0.4525259,0.7097635,-6.049002
APOE,-0.2621224720,11.283661,-1.531338739,0.1343779,0.3865736,-5.203537
AGT,-0.1607024919,10.326358,-0.761144778,0.4515001,0.7088684,-6.047707
PHGDH,-0.1967675195,9.421077,-0.991801290,0.3278772,0.6105966,-5.850759
SOX9,-0.0807298186,9.511944,-0.348192711,0.7297120,0.8856422,-6.273565
ANK2,0.0051783082,10.888753,0.053568163,0.9575737,0.9849498,-6.332401
ATG7,-0.0441806183,7.569452,-0.606557496,0.5479295,0.7798201,-6.151585
PAX6,0.1870164976,4.392259,0.951795064,0.3475123,0.6258207,-5.888461
TBL1X,0.1614914714,7.646890,1.333574832,0.1906731,0.4655767,-5.469995
